In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, roc_curve, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE 


In [ ]:
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

### so no nulls in this data

In [ ]:
sns.pairplot(df,hue = 'Outcome')

In [ ]:
df_c = df.iloc[:,:]
df_c.iloc[:,1:-1]= df_c.iloc[:,1:-1].replace(0,np.NaN)
df_c.isnull().sum()

In [ ]:
df_c.hist(figsize = (20,20))

In [ ]:
df_c.Glucose.fillna(df_c.Glucose.mean(),inplace = True)
df_c.BloodPressure.fillna(df_c.BloodPressure.mean(),inplace = True)

In [ ]:
df_c.SkinThickness.fillna(df_c.SkinThickness.median(),inplace = True)
df_c.Insulin.fillna(df_c.Insulin.median(),inplace = True)
df_c.BMI.fillna(df_c.BMI.median(),inplace = True)

In [ ]:
df_c.hist(figsize = (20,20))

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

df_copy = df_c.iloc[:,:-1]
target = df_c.iloc[:,-1].to_frame()

sns.boxplot(data = df_copy)
df_copy

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_copy, target, 
                                                    test_size = 0.2,
                                                    shuffle = True, 
                                                    random_state = 0)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

X_train.shape

In [ ]:
sm = SMOTE()
X_res, y_res = sm.fit_resample(X_train,y_train)

In [ ]:
# scaler = Normalizer().fit(X_train) #the scaler is fitted to the training set
# normalized_X_train = scaler.transform(X_train) #the scaler is applied to the training set
# normalized_X_test = scaler.transform(X_test) #the scaler is applied to the test set
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_res)                      
X_test = scaler.transform(X_test)
print("X train before Normalization")
print(X_train[0:5])
print("\nX train after Normalization")
print(X_train[0:5])

In [ ]:
df2 = pd.DataFrame(data = np.c_[X_train , y_res] , columns = df.columns )
df2

In [ ]:
sns.pairplot(df2,hue = 'Outcome')

In [ ]:
knn = KNeighborsClassifier(20)
knn.fit(X_train,y_res)
y_exp = knn.predict(X_test)
accuracy_score(y_test, y_exp)

In [ ]:
param_grid = {
                'n_neighbors': np.arange(1, 50, 1)
             }

knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn,param_grid, cv=5)
knn_cv.fit(X_train, y_res)

print("Best Score:" + str(knn_cv.best_score_))
print("Best Parameters: " + str(knn_cv.best_params_))

In [ ]:
y_pred = knn_cv.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
lr = LogisticRegression(random_state=0)

param_grid = \
{
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': list(range(100,800,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}

lr_cv = GridSearchCV(lr, param_grid=param_grid, cv=5, scoring='roc_auc')

lr_cv.fit(X_train, y_res)

print('Config: %s' % lr_cv.best_params_)
print('Best Score: %s' % lr_cv.best_score_)

In [ ]:
y_pred = lr_cv.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
svc = SVC(kernel='rbf')
svc.fit(X_train, y_res)
y_pred = svc.predict(X_test)

In [ ]:
y_pred = svc.predict(X_test)
print(classification_report(y_test, y_pred))
